<a href="https://colab.research.google.com/github/harshad317/NLP/blob/master/W2v_approach_with_3_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import gensim
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem import LancasterStemmer
lc = LancasterStemmer()
from nltk.stem import SnowballStemmer
sb = SnowballStemmer('english')
import gc
from tqdm import tqdm

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, GRU, Dropout, Embedding, Activation, Conv1D
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from keras.layers import Reshape, Flatten
from keras.optimizers import Adam
from keras.layers import concatenate
from keras.models import Model

import re
import string

In [ ]:
train = pd.read_csv('/content/train_2kmZucJ.csv')
test = pd.read_csv('/content/test_oJQbWVk.csv')

In [ ]:
train['tweet'][:20]

0     #fingerprint #Pregnancy Test https://goo.gl/h1...
1     Finally a transparant silicon case ^^ Thanks t...
2     We love this! Would you go? #talk #makememorie...
3     I'm wired I know I'm George I was made that wa...
4     What amazing service! Apple won't even talk to...
5     iPhone software update fucked up my phone big ...
6     Happy for us .. #instapic #instadaily #us #son...
7     New Type C charger cable #UK http://www.ebay.c...
8     Bout to go shopping again listening to music #...
9     Photo: #fun #selfie #pool #water #sony #camera...
10    hey #apple when you make a new ipod dont make ...
11    Ha! Not heavy machinery but it does what I nee...
12    Contemplating giving in to the iPhone bandwago...
13    I just made another crazy purchase lol my theo...
14    @shaqlockholmes @sam_louise1991 the battery is...
15    From #DeepEllum towards #downtown #Dallas. #Bi...
16    Like and Share if you want this 3D Phone Case ...
17    Go crazy !! #iphonesia, #iphone, #instagoo

In [ ]:
text = ''.join(train['tweet'])

In [ ]:
text[:1000]

"#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphoneFinally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCuI'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!iPhone software update fucked up my phone big time Stupid iPhonesHappy for us .. #instapic #instadaily #us #sony #xperia #xperiaZ https://instagram.com/p/z9qGfWlvj7/New Type C charger cable #UK http://www.ebay.co.uk/itm/-/112598674021 … #bay #Amazon #etsy New Year #Rob Cross #Toby Young #EVEMUN #McMafia #Taylor #SPECTRE 2018 #NewYear #Starting 2018 #recipes #technology #SamsungG

In [ ]:
train['tweet'] = train['tweet'].apply(lambda x: x.lower())
test['tweet'] = test['tweet'].apply(lambda x: x.lower())

In [ ]:
#train['tweet'] = train['tweet'].apply(lambda x: re.sub('[^A-Z a-z 0-9]+', '', x))

In [ ]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+';

In [ ]:
train['url_tweet'] = train['tweet'].apply(lambda x: re.sub(url_regex, '', x))
test['url_tweet'] = test['tweet'].apply(lambda x: re.sub(url_regex, '', x))

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
re_punct = re.compile('[%s]' %re.escape(string.punctuation))
train['punct_tweet'] = [re_punct.sub(' ', w) for w in train['url_tweet']]
test['punct_tweet'] = [re_punct.sub(' ', w) for w in test['url_tweet']]

In [ ]:
train['punct_tweet'] = train['punct_tweet'].apply(lambda x: ' '.join(x.split()))
test['punct_tweet'] = test['punct_tweet'].apply(lambda x: ' '.join(x.split()))

In [ ]:
test.head()

,id,tweet,url_tweet,punct_tweet
0,7921,i hate the new #iphone upgrade. won't let me d...,i hate the new #iphone upgrade. won't let me d...,i hate the new iphone upgrade won t let me dow...
1,7922,currently shitting my fucking pants. #apple #i...,currently shitting my fucking pants. #apple #i...,currently shitting my fucking pants apple imac...
2,7923,"i'd like to puts some cd-roms on my ipad, is t...","i'd like to puts some cd-roms on my ipad, is t...",i d like to puts some cd roms on my ipad is th...
3,7924,my ipod is officially dead. i lost all my pict...,my ipod is officially dead. i lost all my pict...,my ipod is officially dead i lost all my pictu...
4,7925,been fighting itunes all night! i only want th...,been fighting itunes all night! i only want th...,been fighting itunes all night i only want the...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer(ngram_range= (1,1))

In [ ]:
vector.fit_transform(train['punct_tweet'])

<7920x18318 sparse matrix of type '<class 'numpy.float64'>'
	with 117488 stored elements in Compressed Sparse Row format>

In [ ]:
 print(vector.get_feature_names())

['00', '000', '002', '004', '0051', '007', '00am', '00pm', '01', '010111', '010315', '011', '01108nartl', '01204', '01634', '01924', '01am', '01amapril', '01amjune', '01ammay', '01pm', '01pmapril', '01pmjune', '02', '02pyow4aag', '03', '035glzwp4r', '04', '0490love', '05', '06', '07', '0716', '0777', '07yilmaz242', '08', '08184820028', '08t', '09', '0bfnzanwty', '0cmcjv1vdj', '0d9dzq0cxo', '0dythia5w0', '0ey7tmubxe', '0eyq6dwgxy', '0fcbzlaj9i', '0ffj5vnrfo', '0g', '0gl7ddfrbk', '0k3k1wc8fk', '0k8h8yuufr', '0ljymldhby', '0mjjnomhpy', '0mp', '0p48k68rac', '0ssqhh798x', '0t4z54vzxg', '0tjig7dfzv', '0vihbaapiq', '0vkcataugw', '0wautkfgi8', '0wqftykprt', '0x', '0yfpeeslbg', '0yk3fbch0c', '0ä1ö', '10', '100', '1000', '10000', '100400', '100happyday', '100happydays', '100likes', '100odd', '101', '102409', '104x50x60', '106', '106finalact', '1080p', '10gb', '10likes', '10mpx', '10pcs', '10th', '10x', '11', '1111', '114', '117', '1180', '118mb', '11h', '11luglio', '11pm', '11september', '12', '

In [ ]:
freq = pd.Series(vector.get_feature_names())

In [ ]:
top = freq[:50]

In [ ]:
rare =freq[-50:]

In [ ]:
train['clean_tweet'] = train['punct_tweet'].apply(lambda x: ' '.join([t for t in x.split() if t not in top]))
test['clean_tweet'] = test['punct_tweet'].apply(lambda x: ' '.join([t for t in x.split() if t not in top]))

In [ ]:
train['clean_tweet'] = train['punct_tweet'].apply(lambda x: ' '.join([t for t in x.split() if t not in rare]))
test['clean_tweet'] = test['punct_tweet'].apply(lambda x: ' '.join([t for t in x.split() if t not in rare]))

In [ ]:
np.max(train['clean_tweet'].apply(lambda x: len(x)))

287

In [ ]:
test.head(2)

,id,tweet,url_tweet,punct_tweet,clean_tweet
0,7921,i hate the new #iphone upgrade. won't let me d...,i hate the new #iphone upgrade. won't let me d...,i hate the new iphone upgrade won t let me dow...,i hate the new iphone upgrade won t let me dow...
1,7922,currently shitting my fucking pants. #apple #i...,currently shitting my fucking pants. #apple #i...,currently shitting my fucking pants apple imac...,currently shitting my fucking pants apple imac...


In [ ]:
train = train.drop(['id', 'tweet', 'url_tweet', 'punct_tweet'], axis = 1)
test = test.drop(['id', 'tweet', 'url_tweet', 'punct_tweet'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(train, test_size = 0.1, random_state = 2020)

In [ ]:
max_features = 17867
embed_size = 300
max_len = 287

In [ ]:
tokenizer = Tokenizer(num_words= max_features)

In [ ]:
tokenizer.fit_on_texts(list(train['clean_tweet']) + list(test['clean_tweet']))
X_train = tokenizer.texts_to_sequences(train['clean_tweet'])
X_test = tokenizer.texts_to_sequences(test['clean_tweet'])
X_train = pad_sequences(X_train, maxlen= max_len)
X_test = pad_sequences(X_test, maxlen= max_len)
y_train = train['label']


In [ ]:
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-10-16 20:07:25--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.99.125
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.99.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.1’

GoogleNews-vectors- 100%[===================>]   1.53G  46.7MB/s    in 41s     

2020-10-16 20:08:07 (38.2 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.1’ saved [1647046227/1647046227]



In [ ]:
#model = gensim.models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
%%time
EMBEDDING_FILE = '/content/drive/My Drive/w2v/glove.6B.300d.txt'
def get_coefs(word,*arr): 
  return word, np.asarray(arr, dtype='float32')
  
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_1 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_1[i] = embedding_vector

del embeddings_index; gc.collect() 

/usr/local/lib/python3.6/dist-packages/IPython/core/magic.py:188: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  call = lambda f, *a, **k: f(*a, **k)


CPU times: user 28 s, sys: 1.36 s, total: 29.3 s
Wall time: 32.9 s


In [ ]:
%%time
from gensim.models import KeyedVectors

EMBEDDING_FILE = '/content/GoogleNews-vectors-negative300.bin.gz'
embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_2 = (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= max_features: continue
    if word in embeddings_index:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix_2[i] = embedding_vector
        
del embeddings_index; gc.collect()     

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 1min 29s, sys: 3.26 s, total: 1min 33s
Wall time: 1min 33s


In [ ]:
%%time
EMBEDDING_FILE = '/content/drive/My Drive/w2v/crawl-300d-2M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_3 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_3[i] = embedding_vector
        
del embeddings_index; gc.collect()

/usr/local/lib/python3.6/dist-packages/IPython/core/magic.py:188: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  call = lambda f, *a, **k: f(*a, **k)


CPU times: user 2min 20s, sys: 14.7 s, total: 2min 35s
Wall time: 2min 45s


In [ ]:
embedding_matrix = np.concatenate((embedding_matrix_1, embedding_matrix_2, embedding_matrix_3), axis=1)  
del embedding_matrix_1, embedding_matrix_2, embedding_matrix_3
gc.collect()
np.shape(embedding_matrix)

(17867, 900)

In [ ]:
inp = Input(shape=(max_len,))
x = Embedding(max_features, embed_size * 3, weights=[embedding_matrix])(inp)
x = Dropout(0.1)(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
x = Dense(16, activation="relu")(conc)
x = Dropout(0.3)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=512, epochs=7)

Epoch 1/7
16/16 [==============================] - 12s 753ms/step - loss: 0.5097 - accuracy: 0.7338
Epoch 2/7
16/16 [==============================] - 12s 743ms/step - loss: 0.3393 - accuracy: 0.7442
Epoch 3/7
16/16 [==============================] - 12s 750ms/step - loss: 0.2941 - accuracy: 0.8850
Epoch 4/7
16/16 [==============================] - 12s 759ms/step - loss: 0.2636 - accuracy: 0.9071
Epoch 5/7
16/16 [==============================] - 12s 769ms/step - loss: 0.2150 - accuracy: 0.9231
Epoch 6/7
16/16 [==============================] - 12s 762ms/step - loss: 0.1984 - accuracy: 0.9274
Epoch 7/7
16/16 [==============================] - 12s 764ms/step - loss: 0.1809 - accuracy: 0.9414


In [ ]:
pred = model.predict([X_test], batch_size=102, verbose=1)

20/20 [==============================] - 1s 56ms/step


In [ ]:
testing = pd.read_csv('/content/test_oJQbWVk.csv')

In [ ]:
submission = pd.DataFrame()
submission['id'] = testing['id']
submission['label'] = (pred>0.51).astype(int)
submission.to_csv('combining.csv', index= False)

In [ ]:
from sklearn import metrics

In [ ]:
thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(y_test, (pred > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh = thresholds[0][0]
print("Best threshold: ", best_thresh)

F1 score at threshold 0.1 is 0.8067226890756303
F1 score at threshold 0.11 is 0.8084210526315789
F1 score at threshold 0.12 is 0.8059071729957806
F1 score at threshold 0.13 is 0.8059071729957806
F1 score at threshold 0.14 is 0.8033826638477801
F1 score at threshold 0.15 is 0.8033826638477801
F1 score at threshold 0.16 is 0.8067940552016986
F1 score at threshold 0.17 is 0.8067940552016986
F1 score at threshold 0.18 is 0.8085106382978723
F1 score at threshold 0.19 is 0.8085106382978723
F1 score at threshold 0.2 is 0.8085106382978723
F1 score at threshold 0.21 is 0.811965811965812
F1 score at threshold 0.22 is 0.8154506437768241
F1 score at threshold 0.23 is 0.8154506437768241
F1 score at threshold 0.24 is 0.8207343412526998
F1 score at threshold 0.25 is 0.8207343412526998
F1 score at threshold 0.26 is 0.8225108225108225
F1 score at threshold 0.27 is 0.8225108225108225
F1 score at threshold 0.28 is 0.8199566160520607
F1 score at threshold 0.29 is 0.8199566160520607
F1 score at threshold 0